# Подготовка: установка latinpipe

In [ ]:
import requests
import os
import tarfile
import pandas as pd
import glob
import subprocess
import tensorflow

In [20]:
# Клонируем репозиторий latinpipe
repo_url = "https://github.com/ufal/evalatin2024-latinpipe.git"
repo_dir = "../evalatin2024-latinpipe"

# Клонируем только если ещё не клонирован
if not os.path.exists(repo_dir):
    subprocess.run(["git", "clone", repo_url, repo_dir])
else:
    print(f"Папка '{repo_dir}' уже существует, пропускаем клонирование.")

# Устанавливаем зависимости
req_path = os.path.join(repo_dir, "requirements.txt")
if os.path.exists(req_path):
    subprocess.run(["pip", "install", "-r", req_path])
else:
    print(f"requirements.txt не найден по пути: {req_path}")

Cloning into '../evalatin2024-latinpipe'...


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


ВАЖНО! в ../evalatin2024-latinpipe/latinpipe_evalatin24.py замените строку 22 на **from typing_extensions import Self**

# Скачиваем модель

In [13]:
model_dir = "../EvaLatin"
os.makedirs(model_dir, exist_ok=True)

file_name = "latinpipe-evalatin24-240520.tar.gz"
file_url = f"https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-5671/{file_name}"
archive_path = os.path.join(model_dir, file_name)

# Скачивание
if not os.path.exists(archive_path):
    response = requests.get(file_url, stream=True)
    with open(archive_path, 'wb') as f:
        for chunk in response.iter_content(8192):
            f.write(chunk)
    print("Архив скачан.")
else:
    print("Архив уже существует, пропускаем скачивание.")

# Распаковка
with tarfile.open(archive_path, "r:gz") as tar:
    tar.extractall(model_dir)
print(f"Архив распакован в {model_dir}.")


Архив скачан.
Архив распакован в ../EvaLatin.


# Лемматизация

In [9]:
!python3.11 latinpipe_evalatin24.py \
    --load ../EvaLatin/latinpipe-evalatin24-240520/model.weights.h5 \
    --exp ../EvaLatin/prediction \
    --test ../data/conllu/*.conllu

2025-05-11 17:07:30.544829: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
tokenizer_config.json: 100%|████████████████████| 354/354 [00:00<00:00, 217kB/s]
vocab.json: 100%|██████████████████████████| 1.43M/1.43M [00:00<00:00, 2.19MB/s]
merges.txt: 100%|████████████████████████████| 994k/994k [00:00<00:00, 3.85MB/s]
tokenizer.json: 100%|██████████████████████| 3.51M/3.51M [00:00<00:00, 19.4MB/s]
config.json: 100%|█████████████████████████████| 676/676 [00:00<00:00, 3.08MB/s]


# Выбираем столбцы с леммами и сохраняем в .csv

In [10]:
input_folder = "../EvaLatin/prediction"
output_folder = "../data/lemmas/"

os.makedirs(output_folder, exist_ok=True)

# Ищем все .conllu файлы в папке
file_list = glob.glob(os.path.join(input_folder, "*.conllu"))

# Индекс столбца, который нужно выделить
column_name = 2

for file_path in file_list:
    df = pd.read_csv(file_path, sep="\t", header=None)
    selected_dataframe = df[[column_name]]

    # Формируем имя выходного файла
    base_name = os.path.basename(file_path)
    new_name = base_name.replace(".conllu", "_лем.csv")
    output_path = os.path.join(output_folder, new_name)

    selected_dataframe.to_csv(output_path, header=False, index=False)

    print(f"Обработан файл: {base_name} -> {new_name}")

Обработан файл: petrus_cantor_distinctiones.predicted.conllu -> petrus_cantor_distinctiones.predicted_лем.csv
Обработан файл: nicolas_de_biard-distinctiones.predicted.conllu -> nicolas_de_biard-distinctiones.predicted_лем.csv
Обработан файл: mauritius_hibernicus_distinctiones.predicted.conllu -> mauritius_hibernicus_distinctiones.predicted_лем.csv
Обработан файл: anonymous_angelus.predicted.conllu -> anonymous_angelus.predicted_лем.csv
Обработан файл: guido_ebroicensis_summa.predicted.conllu -> guido_ebroicensis_summa.predicted_лем.csv
Обработан файл: alanus_de_insula_dist_summa.predicted.conllu -> alanus_de_insula_dist_summa.predicted_лем.csv
Обработан файл: nicolas_de_gorran_distinctiones.predicted.conllu -> nicolas_de_gorran_distinctiones.predicted_лем.csv
